In [ ]:
import streamlit as st
import requests
import json
from datetime import datetime

# Page configuration
st.set_page_config(
    page_title="Health Advice Chatbot",
    page_icon="🏥",
    layout="wide"
)

# Custom CSS
st.markdown("""
    <style>
    .main {
        padding: 2rem;
    }
    .stTextInput > div > div > input {
        background-color: #f0f2f6;
    }
    .chat-message {
        padding: 1.5rem;
        border-radius: 0.5rem;
        margin-bottom: 1rem;
        display: flex;
        flex-direction: column;
    }
    .user-message {
        background-color: #e3f2fd;
        border-left: 5px solid #2196f3;
    }
    .bot-message {
        background-color: #f1f8e9;
        border-left: 5px solid #4caf50;
    }
    .disclaimer {
        background-color: #fff3cd;
        border: 1px solid #ffc107;
        padding: 1rem;
        border-radius: 0.5rem;
        margin-bottom: 1rem;
    }
    </style>
""", unsafe_allow_html=True)

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

if "api_key" not in st.session_state:
    st.session_state.api_key = ""

# Function to call DeepSeek API
def get_deepseek_response(messages, api_key):
    """Call DeepSeek API to get health advice"""
    url = "https://api.deepseek.com/v1/chat/completions"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Add system prompt for health advice
    system_message = {
        "role": "system",
        "content": """You are a helpful health advice assistant. Provide general health information, wellness tips, and lifestyle advice.

IMPORTANT DISCLAIMERS:
- You are NOT a replacement for professional medical advice
- Always remind users to consult healthcare professionals for diagnosis and treatment
- Do not provide specific diagnoses or prescribe medications
- Focus on general wellness, preventive care, and healthy lifestyle tips
- If someone describes serious symptoms, urge them to seek immediate medical attention

Be empathetic, informative, and always prioritize user safety."""
    }

    full_messages = [system_message] + messages

    data = {
        "model": "deepseek-chat",
        "messages": full_messages,
        "temperature": 0.7,
        "max_tokens": 1000
    }

    try:
        response = requests.post(url, headers=headers, json=data, timeout=30)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        return f"Error connecting to DeepSeek API: {str(e)}"
    except (KeyError, IndexError) as e:
        return f"Error parsing API response: {str(e)}"

# Sidebar for API key
with st.sidebar:
    st.title("⚙️ Settings")

    api_key_input = st.text_input(
        "DeepSeek API Key",
        type="password",
        value=st.session_state.api_key,
        help="Enter your DeepSeek API key. Get one at https://platform.deepseek.com"
    )

    if api_key_input:
        st.session_state.api_key = api_key_input
        st.success("✅ API Key configured")
    else:
        st.warning("⚠️ Please enter your API key")

    st.markdown("---")

    st.markdown("### 📋 How to get API Key:")
    st.markdown("""
    1. Visit [DeepSeek Platform](https://platform.deepseek.com)
    2. Sign up/Login
    3. Navigate to API Keys section
    4. Create a new API key
    5. Copy and paste it above
    """)

    st.markdown("---")

    if st.button("🗑️ Clear Chat History"):
        st.session_state.messages = []
        st.rerun()

    st.markdown("---")
    st.markdown("### 📊 Chat Statistics")
    st.metric("Messages", len(st.session_state.messages))

# Main content
st.title("🏥 Health Advice Chatbot")
st.markdown("*Powered by DeepSeek AI*")

# Disclaimer
st.markdown("""
<div class="disclaimer">
    <strong>⚠️ Medical Disclaimer:</strong> This chatbot provides general health information only.
    It is NOT a substitute for professional medical advice, diagnosis, or treatment.
    Always consult with qualified healthcare providers for medical concerns.
</div>
""", unsafe_allow_html=True)

# Display chat messages
chat_container = st.container()
with chat_container:
    for message in st.session_state.messages:
        role = message["role"]
        content = message["content"]

        if role == "user":
            st.markdown(f"""
            <div class="chat-message user-message">
                <strong>👤 You:</strong><br>{content}
            </div>
            """, unsafe_allow_html=True)
        else:
            st.markdown(f"""
            <div class="chat-message bot-message">
                <strong>🤖 Health Assistant:</strong><br>{content}
            </div>
            """, unsafe_allow_html=True)

# Chat input
if st.session_state.api_key:
    user_input = st.chat_input("Ask about health, wellness, nutrition, exercise, or general medical information...")

    if user_input:
        # Add user message
        st.session_state.messages.append({"role": "user", "content": user_input})

        # Get bot response
        with st.spinner("🤔 Thinking..."):
            bot_response = get_deepseek_response(
                st.session_state.messages,
                st.session_state.api_key
            )

        # Add bot message
        st.session_state.messages.append({"role": "assistant", "content": bot_response})

        # Rerun to display new messages
        st.rerun()
else:
    st.info("👈 Please enter your DeepSeek API key in the sidebar to start chatting")

# Sample questions
with st.expander("💡 Sample Questions"):
    st.markdown("""
    - What are some healthy breakfast options?
    - How can I improve my sleep quality?
    - What exercises are good for back pain?
    - How much water should I drink daily?
    - What are the benefits of meditation?
    - How can I reduce stress naturally?
    - What vitamins are important for immune health?
    - Tips for maintaining a healthy weight?
    """)

# Footer
st.markdown("---")
st.markdown("""
<div style="text-align: center; color: #666; padding: 1rem;">
    <small>Built with Streamlit | Powered by DeepSeek AI<br>
    For educational purposes only - Always consult healthcare professionals</small>
</div>
""", unsafe_allow_html=True)